# Climate Profiles 2

### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [9]:
from typing import Tuple

import numpy as np
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm  # Progress bar

import climakitae as ck
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv, retrieve_profile_data
from climakitae.util.warming_levels import get_gwl_at_year
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.core.data_interface import (
    get_data_options,
    get_subsetting_options,
    get_data,
)

import warnings
warnings.filterwarnings("ignore")

# import functions needed for testing
from climakitae.core.constants import UNSET
from climakitae.core.paths import VARIABLE_DESCRIPTIONS_CSV_PATH
from climakitae.explore.typical_meteorological_year import is_HadISD
from climakitae.explore.standard_year_profile import (
    _get_clean_standardyr_filename,
    _check_stations,
    _check_cached_area,
    _check_lat_lon,
    match_str_to_wl,
)
from climakitae.util.utils import read_csv_file
from climakitae.util.warming_levels import get_gwl_at_year

In [2]:
variable = "Air Temperature at 2m"
units = "degF"
qtile = 0.5
warming_levels = [1.5]
no_delta = True

# warming level window size
valid_window = 5
invalid_window_1 = 2
invalid_window_2 = 5.5

# station name options
station_list = [
    "Sacramento Executive Airport (KSAC)",
    "Santa Barbara Municipal Airport (KSBA)",
]
station_name = ["Sacramento Executive Airport (KSAC)"]
custom_name = ["Custom Station Name"]
mixed_list = [
    "Custom Station Name",
    "Santa Barbara Municipal Airport (KSBA)",
]
custom_list = [
    "Custom Name 1",
    "Custom Name 2",
]

# lat lon
# latitude = 34.4041
# longitude = -121.5160 # no data found for these lat and lon bounds!
latitude = 34.19966
longitude = -118.36543  # no data found for these lat and lon bounds!

# cached area
area_name = "Los Angeles County"

### Time-based file modification

Valid inputs to test
- "approach" = "Time" with a "centered_year" between 2015 and 20199
- no "warming_level" input

Invalid inputs to test
1. "approach" = "Time" with a "centered_year" between 2015 and 2099, with "warming_level" input
2. "approach" = "Time" and no "centered_year" provided
3. "centered_year" outside of 2015-2099
4. "approach" that is not "Time" or "Warming Level"

In [ ]:
profile_selections = {
    "variable": variable,
    "resolution": "3 km",
    "q": qtile,
    #"warming_level": [1.5],
    "units": units,
    "no_delta": no_delta,
    # approach
    "approach": "Time",
    "centered_year": 2016,
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    "latitude": (
        latitude - 0.02,
        latitude + 0.02,
    ),  # uncomment for a using a custom coordinate location
    "longitude": (
        longitude - 0.02,
        longitude + 0.02,
    ),  # uncomment for a custom coordinate location
    # "cached_area": area_name, # uncomment for a cached area
}

In [ ]:
profile = get_climate_profile(**profile_selections)

In [ ]:
export_profile_to_csv(profile, **profile_selections)

In [11]:
get_gwl_at_year(1950, "SSP 3-7.0")

Year 1950 before 2015, using Historical data


,5%,Mean,95%
Historical,-0.06,0.25,0.61


In [3]:
downscaling_method = "Dynamical"
resolution = "3 km"
timescale = "hourly"
tidy = True
enable_hidden_vars = False

In [8]:
# try out:
# SSP 2-4.5 ->
# SSP 5-8.5 ->
# SSP 3-7.0 ->

get_data_options(
    variable = None, downscaling_method = downscaling_method, resolution = None, timescale = timescale, scenario="SSP 3-7.0"
)

variable  \
downscaling_method scenario  timescale                                                      
Dynamical          SSP 3-7.0 hourly                            Fosberg fire weather index   
                             hourly     Instantaneous downwelling longwave flux at bottom   
                             hourly     Instantaneous downwelling longwave flux at bottom   
                             hourly     Instantaneous downwelling longwave flux at bottom   
                             hourly     Instantaneous downwelling clear sky longwave f...   
...                                                                                   ...   
                             hourly                               Specific humidity at 2m   
                             hourly                               Specific humidity at 2m   
                             hourly                                       NOAA Heat Index   
                             hourly                                       NOAA Heat Index   
                             hourly                                       NOAA Heat Index   

                                       resolution  
downscaling_method scenario  timescale             
Dynamical          SSP 3-7.0 hourly          3 km  
                             hourly         45 km  
                             hourly          9 km  
                             hourly          3 km  
                             hourly         45 km  
...                                           ...  
                             hourly          9 km  
                             hourly          3 km  
                             hourly         45 km  
                             hourly          9 km  
                             hourly          3 km  

[82 rows x 2 columns]

How...I am so confused. Do SSPs correspond to certain models?

In [42]:
retrieve_params = {
    "variable": variable,
    "resolution": "45 km",
    # "q": qtile,
    "warming_level": [1.14],
    "units": units,
    # "no_delta": False,
    # # approach
    # "approach": "Time",
    # "centered_year": 2016,
    # # warming level window
    "warming_level_window": 5,
    # # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name,  # uncomment for a cached area
}

In [41]:
# try out:
# SSP 2-4.5 ->
# SSP 5-8.5 ->
# SSP 3-7.0 ->

get_gwl_at_year(2016, "SSP 5-8.5")

,5%,Mean,95%
SSP 5-8.5,1.04,1.14,1.25


In [43]:
historic_data, future_data = retrieve_profile_data(**retrieve_params)

In [33]:
future_data['simulation']

<xarray.DataArray 'simulation' (simulation: 10)> Size: 2kB
array(['WRF_CESM2_r11i1p1f1_historical+ssp245',
       'WRF_CESM2_r11i1p1f1_historical+ssp370',
       'WRF_CESM2_r11i1p1f1_historical+ssp585',
       'WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370',
       'WRF_EC-Earth3_r1i1p1f1_historical+ssp370',
       'WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370',
       'WRF_FGOALS-g3_r1i1p1f1_historical+ssp370',
       'WRF_MIROC6_r1i1p1f1_historical+ssp370',
       'WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370',
       'WRF_TaiESM1_r1i1p1f1_historical+ssp370'], dtype='<U44')
Coordinates:
    Lambert_Conformal  int64 8B 0
    centered_year      (simulation) int64 80B 2015 2015 2014 ... 2027 2009 2020
  * simulation         (simulation) <U44 2kB 'WRF_CESM2_r11i1p1f1_historical+...
Attributes:
    description:  combined simulation and scenario

In [19]:
# try out:
# SSP 2-4.5 ->
# SSP 5-8.5 ->
# SSP 3-7.0 ->

centered_year = 2016
scenario = "SSP 2-4.5"

In [21]:
gwl_options = get_gwl_at_year(centered_year, scenario)
[float(gwl_options.loc[scenario, "Mean"])]

[1.13]